# Mass Transfer (MT)
March 17th, 2025
Data analysis by Alex Quemel, Rahil Shaik, Sneha Kancharla, and Ethan Mibu

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
from scipy.stats import linregress
import math
import warnings
warnings.filterwarnings('ignore')

### Reading in Data from .csv

In [ ]:
df = pd.read_csv("MW1-Mass Transfer Lab Raw Data - Sheet1.csv")

In [ ]:
df

## Analysis Question 1
Calculate measured evaporation rates as mass fluxes using units of kg H2O/m²·s.

In [8]:
# placeholder values (will import dataframe when ready)
delta_m_water = 0.1 # kg (replace)
area_tray = 0.555 * 0.200 #m^2 (actual value)
time_elapsed = 300 # seconds (replace)

evaporative_flux = delta_m_water / (area_tray * time_elapsed)
evaporative_flux

0.003003003003003003

## Analysis Question 2
Use Eqs. (5) and (6) and appropriate fluid properties to calculate an average for the concentration and the temperature gradients at the evaporating surface normalized along the length of the entire plate (ignoring at this point that the gradients are functions of position along the length of the pan).

In [ ]:
k_air = 0.026  # Thermal conductivity of air (W/m·K)
hv = 2.26e6  # Heat of vaporization of water (J/kg)
D_AB = 2.5e-5  # Diffusivity of water in air (m²/s)
R = 8.314  # Gas constant (J/mol·K)
P_air = 1.0e5  # Atmospheric pressure (Pa)
T_air = 300  # Bulk air temperature (K)
MW_H2O = 18.015  # Molar mass of water (g/mol)

# Experimental data inputs
mass_flux = np.array([0.0005, 0.0008, 0.0012]) # replace with df values
air_velocity = np.array([1.5, 3.0, 4.5]) # replace with df values

# Convert mass flux to molar flux J_A (mol/m²·s)
J_A = (mass_flux / MW_H2O) * 1000  # Convert kg to g and divide by molar mass

# Calculate temperature gradient (dT/dy) using Eq. (5)
dT_dy = (hv * J_A) / k_air  # K/m

# Calculate total molar concentration of air (c_total = P / RT)
c_total = P_air / (R * T_air)  # mol/m³

# Calculate concentration gradient (dx_A/dy) using Eq. (6)
dxA_dy = J_A / (D_AB * c_total)  # 1/m

# # Organize results into a DataFrame
# df_results = pd.DataFrame({
#     'Air Velocity (m/s)': air_velocity,
#     'Mass Flux (kg/m²·s)': mass_flux,
#     'Molar Flux J_A (mol/m²·s)': J_A,
#     'Temperature Gradient dT/dy (K/m)': dT_dy,
#     'Concentration Gradient dxA/dy (1/m)': dxA_dy
# })

# Plot temperature gradient vs. air velocity
# plt.figure(figsize=(8, 6))
# plt.plot(air_velocity, dT_dy, marker='o', linestyle='-', label='dT/dy (K/m)')
# plt.xlabel('Air Velocity (m/s)')
# plt.ylabel('Temperature Gradient (K/m)')
# plt.title('Temperature Gradient vs Air Velocity')
# plt.legend()
# plt.grid(True)
# plt.show()

# Plot concentration gradient vs. air velocity
# plt.figure(figsize=(8, 6))
# plt.plot(air_velocity, dxA_dy, marker='s', linestyle='-', label='dxA/dy (1/m)', color='r')
# plt.xlabel('Air Velocity (m/s)')
# plt.ylabel('Concentration Gradient (1/m)')
# plt.title('Concentration Gradient vs Air Velocity')
# plt.legend()
# plt.grid(True)
# plt.show()

NameError: name 'mass_flux' is not defined

## Analysis Question 3
Using the gradients calculated above, estimate the average thicknesses of the concentration and thermal boundary layers by assuming that both the concentration and the temperature profiles are linear starting at the interface value and ending at the bulk flow values at the top boundary. Which boundary-layer thickness is larger? Is the relationship between the two boundary layer thicknesses as expected? How do the average boundary layer thicknesses scale with velocity? Is this as expected? Why or why not?

In [ ]:
# Given or assumed experimental conditions
T_surface = 310  # K (Example: Water surface temperature)
T_bulk = 300  # K (Example: Bulk air temperature)
xA_surface = 0.03  # Water vapor mole fraction at surface (Example from Antoine Equation)
xA_bulk = 0.005  # Water vapor mole fraction in bulk air (Example from humidity data)

# Compute temperature and concentration differences
delta_T = T_surface - T_bulk  # Temperature difference (K)
delta_xA = xA_surface - xA_bulk  # Mole fraction difference

# Compute boundary layer thicknesses
delta_T_layer = delta_T / dT_dy  # Thermal boundary layer thickness (m)
delta_C_layer = delta_xA / dxA_dy  # Concentration boundary layer thickness (m)

# Organize results
df_results = pd.DataFrame({
    'Air Velocity (m/s)': air_velocity,
    'Thermal Boundary Layer δ_T (m)': delta_T_layer,
    'Concentration Boundary Layer δ_C (m)': delta_C_layer
})

# Display results
tools.display_dataframe_to_user(name="Boundary Layer Thicknesses", dataframe=df_results)

# Plot boundary layer thickness vs velocity
plt.figure(figsize=(8, 6))
plt.plot(air_velocity, delta_T_layer, marker='o', linestyle='-', label='Thermal Boundary Layer (δ_T)')
plt.plot(air_velocity, delta_C_layer, marker='s', linestyle='-', label='Concentration Boundary Layer (δ_C)', color='r')
plt.xlabel('Air Velocity (m/s)')
plt.ylabel('Boundary Layer Thickness (m)')
plt.title('Boundary Layer Thickness vs Air Velocity')
plt.legend()
plt.grid(True)
plt.show()

# Check scaling with velocity using linear regression on log-log plot
log_V = np.log(air_velocity)
log_delta_T = np.log(delta_T_layer)
log_delta_C = np.log(delta_C_layer)

slope_T, intercept_T, _, _, _ = linregress(log_V, log_delta_T)
slope_C, intercept_C, _, _, _ = linregress(log_V, log_delta_C)

print(f"Scaling relationship for δ_T: δ_T ∝ V^({slope_T:.2f})")
print(f"Scaling relationship for δ_C: δ_C ∝ V^({slope_C:.2f})")


## Analysis Question 4
How does km, the average mass transfer coefficient along the length of the plate, vary with the velocity of the gas stream?  Is the scaling of km with velocity what you would expect?  Why or why not?  Is this scaling consistent with the laminar/turbulent nature of the air flow in the experiment, as calculated from the Reynolds number? That is to say, the Reynolds number controls the laminar/turbulent nature of flow, but this nature of the flow affects the scaling of km with velocity. Is there self-consistency between this scaling and the Reynolds number?

In [ ]:
# Compute mass transfer coefficient from experimental data
km_exp = J_A / delta_xA  # Experimental km (m/s)

# Compute mass transfer coefficient from film theory
km_film = D_AB / delta_C_layer  # Film theory km (m/s)

# Compute Reynolds number
Re = (rho_air * air_velocity * L_char) / mu_air  # Reynolds number

# Organize results into a DataFrame
df_results = pd.DataFrame({
    'Air Velocity (m/s)': air_velocity,
    'Reynolds Number': Re,
    'Experimental km (m/s)': km_exp,
    'Film Theory km (m/s)': km_film
})

# Display results
tools.display_dataframe_to_user(name="Mass Transfer Coefficient Analysis", dataframe=df_results)

# Plot km vs air velocity (log-log plot)
plt.figure(figsize=(8, 6))
plt.loglog(air_velocity, km_exp, marker='o', linestyle='-', label='Experimental km')
plt.loglog(air_velocity, km_film, marker='s', linestyle='--', label='Film Theory km', color='r')
plt.xlabel('Air Velocity (m/s)')
plt.ylabel('Mass Transfer Coefficient km (m/s)')
plt.title('Mass Transfer Coefficient vs Air Velocity')
plt.legend()
plt.grid(True)
plt.show()

# Perform linear regression for scaling (log-log)
log_V = np.log(air_velocity)
log_km_exp = np.log(km_exp)
log_km_film = np.log(km_film)

slope_exp, intercept_exp, _, _, _ = linregress(log_V, log_km_exp)
slope_film, intercept_film, _, _, _ = linregress(log_V, log_km_film)

print(f"Experimental scaling: km ∝ V^({slope_exp:.2f})")
print(f"Film theory scaling: km ∝ V^({slope_film:.2f})")